In [3]:
import numpy as np
import pandas as pd
import pypsa
import pypsa.networkclustering as pync
import datetime
from six import iteritems
import os 
import sys  

file_name =  os.path.basename(sys.argv[0])
directory = os.path.dirname(os.path.realpath(file_name))+'/Output'
n_file = 'Output/' + [s for s in os.listdir(directory) if 'EU_Network_created' in s][0]

#Load Network
n = pypsa.Network()
n.import_from_netcdf(n_file)

INFO:pypsa.io:Imported network EU_Network_created_25102020.nc has buses, carriers, generators, lines, links, loads, storage_units


In [3]:
#Temporal aggregation - reduce to one timestep every n hours
hours = '120H'

# code below borrowed from pypsa-eur prepare_network.py, average_every_nhours function
m = n.copy(with_time=False)

snapshot_weightings = n.snapshot_weightings.resample(hours).sum()
m.set_snapshots(snapshot_weightings.index)
m.snapshot_weightings = snapshot_weightings

for c in n.iterate_components():
    pnl = getattr(m, c.list_name+"_t")
    for k, df in iteritems(c.pnl):
        if not df.empty:
            pnl[k] = df.resample(hours).mean()

In [4]:
#Solve network over all timesteps. WARNING: Takes 36-48 hours to solve
print(datetime.datetime.now())
m.lopf(solver_name='gurobi')
print(datetime.datetime.now())

m.export_to_netcdf('Output/storage_120H.nc')

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1632792_copy
  Lower bound: 6857634121893.111
  Upper bound: 6857634121893.111
  Number of objectives: 1
  Number of constraints: 1048062
  Number of variables: 1632792
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 1632792
  Number of nonzeros: 4557464
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Term

(<SolverStatus.ok: 'ok'>, <TerminationCondition.optimal: 'optimal'>)

In [10]:
#Interpolate storage values between 120 hour mark and save as csv
dates = pd.DataFrame(index=n.loads_t.p.index)
m.storage_units_t.state_of_charge.index = m.storage_units_t.state_of_charge.index + pd.Timedelta('+119 hours')
m.storage_units_t.state_of_charge = pd.concat([dates,m.storage_units_t.state_of_charge],axis=1)
m.storage_units_t.state_of_charge = pd.concat([pd.DataFrame(m.storage_units_t.state_of_charge.iloc[-1]).T,m.storage_units_t.state_of_charge]).interpolate().iloc[1:]

m.storage_units_t.state_of_charge.to_csv('Output/n_storage.csv')
pd.read_csv('Output/n_storage.csv',index_col=0)